### ADD Brand List of Shopinng Mall

In [1]:
import sys

sys.path.append("../../")

In [1]:
# in Jupyter
import re
import sys
import os
from random import randint
from typing import List, Dict, Any, Callable

import asyncio 
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, Page, TimeoutError as PlaywrightTimeoutError 



pw = await async_playwright().start()
browser = await pw.firefox.launch(headless=False)
page = await browser.new_page()


page.set_default_timeout(15000)

async def _load_page( page: Page, url):
    """페이지 로드"""
    await asyncio.sleep(1)
    await page.goto(url)
    await page.wait_for_load_state(state="networkidle")
    await page.wait_for_timeout(1000)
    return page

async def load_cookies(page):
    # Load cookies from a file or database
    # In this example, cookies are loaded from a file named 'cookies.json'
    
    with open('../../router/dev/kream/cookie/cookies.json', 'r') as file:
        cookies = eval(file.read())

    # Set cookies in the current page
    await page.context.add_cookies(cookies)

await load_cookies(page)

In [3]:
async def load_cookies(page):
    # Load cookies from a file or database
    # In this example, cookies are loaded from a file named 'cookies.json'
    
    with open('../../router/dev/kream/cookie/cookies.json', 'r') as file:
        cookies = eval(file.read())

    # Set cookies in the current page
    await page.context.add_cookies(cookies)

await load_cookies(page)

### shop

In [38]:
url = "https://www.consortium.co.uk/adidas-originals-adimatic-panton-core-black-gum-3-hq6903.html"

page = await _load_page(page, url)

In [39]:
size_query= await page.query_selector_all('div.input-box>select > option')

size_list= ([await s.inner_text() for s in size_query ])
size_list = [size for size in size_list if 'Out of Stock' not in size][1:]

In [40]:
size_list

['UK 4',
 'UK 7',
 'UK 8',
 'UK 8.5',
 'UK 9',
 'UK 9.5',
 'UK 10',
 'UK 10.5',
 'UK 11',
 'UK 12']

In [41]:
product_id_text = await page.query_selector('//*[@id="detailsPanel"]/div')
product_id_text = await product_id_text.inner_text()

In [48]:
text = product_id_text.lower()
if "product code" in text :
    product_id = text.split("product code")[1].strip()
else:
    product_id = '-'

product_id.upper()

'-'

### kream

In [2]:
await _load_page(page, "https://kream.co.kr/login")

<Page url='https://kream.co.kr/login'>

In [4]:
async def save_cookies(page):
    # Get cookies from the current page
    cookies = await page.context.cookies()

    # Save cookies to a file or database
    # In this example, cookies are saved to a file named 'cookies.json'
    with open('../../router/dev/kream/cookie/cookies.json', 'w') as file:
        file.write(str(cookies))


In [3]:
async def load_cookies(page):
    # Load cookies from a file or database
    # In this example, cookies are loaded from a file named 'cookies.json'
    
    with open('../../router/dev/kream/cookie/cookies.json', 'r') as file:
        cookies = eval(file.read())

    # Set cookies in the current page
    await page.context.add_cookies(cookies)

await load_cookies(page)
await page.goto("https://kream.co.kr")

<Response url='https://kream.co.kr/' request=<Request url='https://kream.co.kr/' method='GET'>>

In [4]:
brand = await page.query_selector(".wish_count_num")
brand = await brand.inner_text()
int(brand.replace(",",""))

117

In [15]:
x = await page.query_selector_all(".body_list")
volumes = [BeautifulSoup(await i.inner_html()) for i in x]

In [ ]:
# def _extract_volume_data_from(body_list: List) -> List[List]:
#     """
#     크롤링 결과 리스트에서 거래량 추출

#     Args:
#         body_list (List): 크롤링 결과 리스트

#     Returns:
#         List: 크롤링 결과 리스트로 반환
#     """

#     def preprocess_items(tags) -> List:
#         l = []
#         for tag in tags.find_all(class_="list_txt"):
#             l.append(tag.text.lstrip().rstrip())

#         if "빠른" in l[1]:
#             l = l[:1] + l[1].split(" ")[:1] + [True] + l[2:3].replace("빠른배송", "").strip()
#         else:
#             l = l[:2] + [False] + l[2:3].replace("빠른배송", "").strip()
#         return l

#     return [preprocess_items(tags) for tags in body_list]

# body_list = _extract_volume_data_from(volumes)

In [19]:
x = await page.query_selector_all(".body_list")
volumes = [BeautifulSoup(await i.inner_html()) for i in x]


def _extract_volume_data_from(body_list: List) -> List[List]:
    """
    크롤링 결과 리스트에서 거래량 추출

    Args:
        body_list (List): 크롤링 결과 리스트

    Returns:
        List: 크롤링 결과 리스트로 반환
    """

    def preprocess_items(tags) -> List:
        x = []
        for tag in tags.find_all(class_="list_txt"):
            x.append(tag.text.strip())
        if "빠"in x[1] :
            return [x[0], x[1], True, x[2].replace('빠른배송','').strip()]
        else :
            return [x[0], x[1], False, x[2].replace('빠른배송','').strip()]

    return [preprocess_items(tags) for tags in body_list]

body_list = _extract_volume_data_from(volumes)
body_list

[['EU 43', '445,000원', False, '23/11/14'],
 ['EU 41', '436,000원', False, '23/11/13'],
 ['EU 43', '450,000원', False, '23/11/13'],
 ['EU 43', '498,000원', False, '23/11/13'],
 ['EU 42', '436,000원', False, '23/11/13'],
 ['EU 41', '350,000원', False, '23/11/10'],
 ['EU 42', '419,000원', False, '23/11/09'],
 ['EU 41', '419,000원', False, '23/11/08'],
 ['EU 42', '450,000원', False, '23/11/08'],
 ['EU 44', '458,000원', False, '23/11/08'],
 ['EU 42', '423,000원', False, '23/11/08'],
 ['EU 42', '385,000원', False, '23/11/07'],
 ['EU 44', '386,000원', False, '23/11/07'],
 ['EU 40', '369,000원', False, '23/11/07'],
 ['EU 43', '549,000원', False, '23/11/07'],
 ['EU 43', '449,000원', False, '23/11/07'],
 ['EU 42', '400,000원', False, '23/11/06'],
 ['EU 41', '419,000원', False, '23/11/04'],
 ['EU 41', '459,000원', False, '23/11/01'],
 ['EU 41', '408,000원', False, '23/11/01'],
 ['EU 43', '549,000원', False, '23/10/31'],
 ['EU 43', '468,000원', False, '23/10/31'],
 ['EU 43', '460,000원', False, '23/10/30'],
 ['EU 43', 